In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np

file_path = "pai_all_post.csv"
data = pd.read_csv(file_path)
util_list=data['gpu_wrk_util'].to_list()
start_time_list=data['start_time_t'].to_list()
mem_list=data['max_gpu_wrk_mem'].to_list()
duration_list=data['runtime'].to_list()

# Burst days 10-11, 20-21, 30-31, 40-41, 50-51

In [ ]:
first_time = start_time_list[0]
burst_level = 80000
sample_ind_burst = []
burst_days = [(10,11), (20,21), (30,31), (40,41), (50,51)]

print("Original Ratio:")
for day in burst_days:
    left, right = day
    infer_ind = []
    train_ind = []
    for i,t in enumerate(start_time_list):
        if ((start_time_list[i]-first_time)>=left*24*3600) and ((start_time_list[i]-first_time)<right*24*3600):
            if (duration_list[i] <= 50000) and (util_list[i] <= 20): #2:1:14000 #3:1 50,000
                infer_ind.append(i)
            else:
                train_ind.append(i)
    print("%d-%d Num infer %d, num train %d, infer:train %.2f"%(left, right, len(infer_ind), len(train_ind), len(infer_ind)/len(train_ind)))
    inc_samples = burst_level-len(infer_ind) #burst_level*len(train_ind)-len(infer_ind)
    sample_ind = np.random.choice(np.array(infer_ind), size=inc_samples, replace=True, p=None)
    sample_ind_burst.append(sample_ind)


data_days = data.copy()
post_util_list = data_days['gpu_wrk_util'].to_list()
post_start_time_list = data_days['start_time_t'].to_list()
post_mem_list = data_days['max_gpu_wrk_mem'].to_list()
post_duration_list = data_days['runtime'].to_list()
post_job_name_list = data_days['job_name'].to_list()
post_num_inst_list = data_days['inst_num'].to_list()
# print(len(post_start_time_list))
# print(len(post_util_list))
for inds in sample_ind_burst:
    for ind in inds:
        post_util_list.append(util_list[ind])
        post_start_time_list.append(start_time_list[ind])
        post_mem_list.append(mem_list[ind])
        post_duration_list.append(duration_list[ind])
        post_job_name_list.append(job_name_list[ind])
        post_num_inst_list.append(num_inst_list[ind])

print("\nAfter duplicating:")
all_days = [(0,10), (10,11), (11,20), (20,21), (21,30), (30,31), (31,40), (40,41), (41,50), (50,51)]
for day in all_days:
    left, right = day
    infer_cnt = 0
    train_cnt = 0
    for i in range(len(post_start_time_list)):
        if ((post_start_time_list[i]-first_time)>=left*24*3600) and ((post_start_time_list[i]-first_time)<right*24*3600):
            if (post_duration_list[i] <= 50000) and (post_util_list[i] <= 20):
                infer_cnt = infer_cnt+1
            else:
                train_cnt = train_cnt+1
    print("%d-%d, Num infer %d, num train %d, infer:train %.2f"%(left, right, infer_cnt, train_cnt, infer_cnt/train_cnt))
data_burst = {'gpu_wrk_util': post_util_list, 
              'start_time_t': post_start_time_list,
              'max_gpu_wrk_mem': post_mem_list,
              'runtime': post_duration_list,
              'job_name': post_job_name_list,
              'inst_num': post_num_inst_list} 
data_burst = pd.DataFrame(data_burst) 
# file_path = 'pai_all_burst_3_%dx5.csv'%(burst_level)
data_burst.to_csv(file_path)

In [ ]:
all_days = range(0, 57)
train_cnts = []
infer_cnts = []
for day in all_days:
    left = day
    right = left+1
    infer_cnt = 0
    train_cnt = 0
    for i in range(len(post_start_time_list)):
        if ((post_start_time_list[i]-first_time)>=left*24*3600) and ((post_start_time_list[i]-first_time)<right*24*3600):
            if (post_duration_list[i] <= 50000) and (post_util_list[i] <= 20):
                infer_cnt = infer_cnt+1
            else:
                train_cnt = train_cnt+1
    train_cnts.append(train_cnt)
    infer_cnts.append(infer_cnt)

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
fig.set_size_inches(5, 2.5)
line1, = ax.plot(np.arange(0,len(train_cnts)),train_cnts, color='#00B6FF', linewidth=2, label='Training Jobs')
line2, = ax.plot(np.arange(0,len(infer_cnts)),infer_cnts, color='#FF6C00', linewidth=1.5,label='Inference Jobs')
lns = [line1, line2]
labs = [l.get_label() for l in lns]
plt.legend(lns, labs, loc='center',fancybox=False, 
               shadow=False, ncol=2, frameon=False,columnspacing=1.2, labelspacing=0.2, alignment='right',
           bbox_to_anchor=(0.5, 0.05))
# plt.legend()
plt.xlim([0, 60])
plt.xlabel('Time (days)')
plt.ylabel('Number of Jobs')
plt.savefig('figs/num_jobs_57days.pdf', dpi=300)